# Translate `dzn` to `smt2` for OptiMathSAT

### Check Versions of Tools

In [1]:
import subprocess

output = subprocess.check_output("../optimathsat/bin/optimathsat -version", shell=True, universal_newlines=True)
output

'MathSAT5 version 1.6.4 (d39a648d6bb7) (Feb 25 2020 14:02:07, gmp 6.1.2, gcc 9.2.1, 64-bit)\n'

In [2]:
output = subprocess.check_output("../minizinc/build/minizinc --version", shell=True, universal_newlines=True)
output

'MiniZinc to FlatZinc converter, version 2.3.2\nCopyright (C) 2014-2020 Monash University, NICTA, Data61\n'

First generate the FlatZinc files using the MiniZinc tool. Make sure that a `smt2` folder is located inside `./minizinc/share/minizinc/`. Else, to enable OptiMathSAT's support for global constraints download the [smt2.tar.gz](http://optimathsat.disi.unitn.it/data/smt2.tar.gz) package and unpack it there using

```zsh
tar xf smt2.tar.gz $MINIZINC_PATH/share/minizinc/
```

If next output shows a list of `.mzn` files, then this dependency is satified.

In [3]:
output = subprocess.check_output("ls -la ../minizinc/share/minizinc/smt2/", shell=True, universal_newlines=True)
print(output)

total 292
drwxr-xr-x  2 kw kw 4096 Jan 15  2018 .
drwxr-xr-x 11 kw kw 4096 Jun  2 12:31 ..
-rw-r--r--  1 kw kw  328 Nov 13  2017 alldifferent_except_0.mzn
-rw-r--r--  1 kw kw  382 Nov 13  2017 all_different_int.mzn
-rw-r--r--  1 kw kw  396 Nov 13  2017 all_different_set.mzn
-rw-r--r--  1 kw kw  270 Nov 13  2017 all_disjoint.mzn
-rw-r--r--  1 kw kw  150 Nov 14  2017 all_equal_int.mzn
-rw-r--r--  1 kw kw  164 Nov 13  2017 all_equal_set.mzn
-rw-r--r--  1 kw kw  351 Nov 13  2017 among.mzn
-rw-r--r--  1 kw kw  305 Nov  8  2017 arg_max_float.mzn
-rw-r--r--  1 kw kw  291 Nov  8  2017 arg_max_int.mzn
-rw-r--r--  1 kw kw  306 Nov  8  2017 arg_min_float.mzn
-rw-r--r--  1 kw kw  291 Nov  8  2017 arg_min_int.mzn
-rw-r--r--  1 kw kw  480 Nov 13  2017 at_least_int.mzn
-rw-r--r--  1 kw kw  506 Nov 14  2017 at_least_set.mzn
-rw-r--r--  1 kw kw  340 Nov 13  2017 at_most1.mzn
-rw-r--r--  1 kw kw  474 Nov 13  2017 at_most_int.mzn
-rw-r--r--  1 kw kw  502 Nov 13  2017 at_most_set.mzn
-rw-r--r--  1 kw kw 1

## Transform `dzn` to `fzn` Using a Model `mzn`

Then transform the desired `.dzn` file to `.fzn` using a `Mz.mzn` MiniZinc model.

First list all `dzn` files contained in the `dzn_path` that should get processed.

In [5]:
import os

dzn_files = []
dzn_path = 'dzn-files/'

for filename in os.listdir(dzn_path):
    if filename.endswith(".dzn"):
        dzn_files.append(filename)
len(dzn_files)

278

#### Model $Mz_1$

In [7]:
import sys

fzn_path = 'fzn-files/smt2/Mz1-noAbs/'
minizinc_base_cmd = '../minizinc/build/minizinc \
    -Werror \
    --compile --solver org.minizinc.mzn-fzn \
    --search-dir /home/kw/minizinc/share/minizinc/smt2/ \
    mzn-model/Mz1-noAbs.mzn '
translate_count = 0
for dzn in dzn_files:
    translate_count += 1
    minizinc_transform_cmd = minizinc_base_cmd + dzn_path + dzn \
        + ' --output-to-file ' + fzn_path + dzn.replace('.', '-') + '.fzn'
    print(f'''\r({translate_count}/{len(dzn_files)}) Translating {dzn_path + dzn} to {fzn_path + dzn.replace('.', '-')}.fzn''', end='')
    sys.stdout.flush()
    subprocess.check_output(minizinc_transform_cmd, shell=True, 
                                     universal_newlines=True)

(278/278) Translating dzn-files/A033.dzn to fzn-files/smt2/Mz1-noAbs/A033-dzn.fzn

#### Model $Mz_2$

In [8]:
import sys

fzn_path = 'fzn-files/smt2/Mz2-noAbs/'
minizinc_base_cmd = '../minizinc/build/minizinc \
    -Werror \
    --compile --solver org.minizinc.mzn-fzn \
    --search-dir /home/kw/minizinc/share/minizinc/smt2/ \
    mzn-model/Mz2-noAbs.mzn '
translate_count = 0
for dzn in dzn_files:
    translate_count += 1
    minizinc_transform_cmd = minizinc_base_cmd + dzn_path + dzn \
        + ' --output-to-file ' + fzn_path + dzn.replace('.', '-') + '.fzn'
    print(f'''\r({translate_count}/{len(dzn_files)}) Translating {dzn_path + dzn} to {fzn_path + dzn.replace('.', '-')}.fzn''', end='')
    sys.stdout.flush()
    subprocess.check_output(minizinc_transform_cmd, shell=True, 
                                     universal_newlines=True)

(278/278) Translating dzn-files/A033.dzn to fzn-files/smt2/Mz2-noAbs/A033-dzn.fzn

## Translate `fzn` to `smt2`

The generated `.fzn` file can be used to generate a `.smt2` file using the `fzn2smt2.py` script from this [project](https://github.com/PatrickTrentin88/fzn2omt).

#### $Mz_1$

In [9]:
import os

fzn_files = []
fzn_path = 'fzn-files/smt2/Mz1-noAbs/'

for filename in os.listdir(fzn_path):
    if filename.endswith(".fzn"):
        fzn_files.append(filename)
len(fzn_files)

278

In [10]:
smt2_path = 'smt2-files/optimathsat/Mz1-noAbs/'
fzn2smt2_base_cmd = '../fzn2omt/bin/fzn2optimathsat.py '
translate_count = 0
my_env = os.environ.copy()
my_env['PATH'] = "/home/kw/optimathsat/bin:" + my_env['PATH']
for fzn in fzn_files:
    translate_count += 1
    fzn2smt2_transform_cmd = fzn2smt2_base_cmd + fzn_path + fzn \
            + ' --smt2 ' + smt2_path + fzn.replace('.', '-') + '.smt2'
    print(f'''\r({translate_count}/{len(fzn_files)}) Translating {fzn_path + fzn} to {smt2_path + fzn.replace('.', '-')}.smt2''', end='')
    try:
        output = subprocess.check_output(fzn2smt2_transform_cmd, shell=True, env=my_env,
                                     universal_newlines=True, stderr=subprocess.STDOUT)
    except Exception as e:
        output = str(e.output)
    print(f'''\r{output}''', end='')
    sys.stdout.flush()

error: failed to generate SMT-LIB formula. bs/R001-dzn.fzn to smt2-files/optimathsat/Mz1-noAbs/R001-dzn-fzn.smt2
error: failed to generate SMT-LIB formula. Abs/R002-dzn.fzn to smt2-files/optimathsat/Mz1-noAbs/R002-dzn-fzn.smt2


#### $Mz_2$

In [11]:
import os

fzn_files = []
fzn_path = 'fzn-files/smt2/Mz2-noAbs/'

for filename in os.listdir(fzn_path):
    if filename.endswith(".fzn"):
        fzn_files.append(filename)
len(fzn_files)

278

In [12]:
smt2_path = 'smt2-files/optimathsat/Mz2-noAbs/'
fzn2smt2_base_cmd = '../fzn2omt/bin/fzn2optimathsat.py '
translate_count = 0
my_env = os.environ.copy()
my_env['PATH'] = "/home/kw/optimathsat/bin:" + my_env['PATH']
for fzn in fzn_files:
    translate_count += 1
    fzn2smt2_transform_cmd = fzn2smt2_base_cmd + fzn_path + fzn \
            + ' --smt2 ' + smt2_path + fzn.replace('.', '-') + '.smt2'
    print(f'''\r({translate_count}/{len(fzn_files)}) Translating {fzn_path + fzn} to {smt2_path + fzn.replace('.', '-')}.smt2''', end='')
    try:
        output = subprocess.check_output(fzn2smt2_transform_cmd, shell=True, env=my_env,
                                     universal_newlines=True, stderr=subprocess.STDOUT)
    except Exception as e:
        output = str(e.output)
    print(f'''\r{output}''', end='')
    sys.stdout.flush()

error: failed to generate SMT-LIB formula. bs/R001-dzn.fzn to smt2-files/optimathsat/Mz2-noAbs/R001-dzn-fzn.smt2
error: failed to generate SMT-LIB formula. Abs/R002-dzn.fzn to smt2-files/optimathsat/Mz2-noAbs/R002-dzn-fzn.smt2


### Adjust `smt2` Files According to 7.5 in Paper

- Add lower and upper bounds for the decision variable `pfc`
- Add number of cavities as comments for later solution extraction (workaround)

In [13]:
import os
import re

def adjust_smt2_file(smt2_path: str, file: str, write_path: str):

    with open(smt2_path+'/'+file, 'r+') as myfile:
        data = "".join(line for line in myfile)

    filename = os.path.splitext(file)[0]
    print(filename)

    newFile = open(os.path.join(write_path, filename +'_v3.smt2'),"w+")
    newFile.write(data)
    newFile.close()

    openFile = open(os.path.join(write_path, filename +'_v3.smt2'))
    data = openFile.readlines()
    additionalLines = data[-6:]
    data = data[:-6]
    openFile.close()

    newFile = open(os.path.join(write_path, filename +'_v3.smt2'),"w+")
    newFile.writelines([item for item in data])
    newFile.close()

    with open(os.path.join(write_path, filename +'_v3.smt2'),"r") as myfile:
        data = "".join(line for line in myfile)
    newFile = open(os.path.join(write_path, filename +'_v3.smt2'),"w+")
    matches = re.findall(r'\(define-fun .\d\d \(\) Int (\d+)\)', data)
    try:
        cavity_count = int(matches[0])
        print(f'''cavity_count: {cavity_count}''')
        newFile.write(f''';; k={cavity_count}\n''')
        newFile.write(f''';; Extract pfc from\n''')
        for i in range(0,cavity_count):
            newFile.write(f''';; X_INTRODUCED_{str(i)}_\n''')
        newFile.write(data)
        for i in range(1,cavity_count+1):
            lb = f'''(define-fun lbound{str(i)} () Bool (> X_INTRODUCED_{str(i-1)}_ 0))\n'''
            ub = f'''(define-fun ubound{str(i)} () Bool (<= X_INTRODUCED_{str(i-1)}_ {str(cavity_count)}))\n'''
            assertLb = f'''(assert lbound{str(i)})\n'''
            assertUb = f'''(assert ubound{str(i)})\n'''

            newFile.write(lb)
            newFile.write(ub)
            newFile.write(assertLb)
            newFile.write(assertUb)
    except:
        print(f'''Check {filename} for completeness - data missing?''')

    #newFile.writelines([item for item in additionalLines])
    newFile.close()

#### $Mz_1$

In [14]:
smt2_path = 'smt2-files/optimathsat/Mz1-noAbs'

for file in os.listdir(smt2_path):
    if file.endswith(".smt2"):
        adjust_smt2_file(smt2_path=smt2_path, file=file, write_path=f'''{smt2_path}/v4''')

R075-dzn-fzn
cavity_count: 10
R200-dzn-fzn
cavity_count: 48
R014-dzn-fzn
cavity_count: 20
A022-dzn-fzn
cavity_count: 80
A021-dzn-fzn
cavity_count: 80
R145-dzn-fzn
cavity_count: 24
R019-dzn-fzn
cavity_count: 64
R177-dzn-fzn
cavity_count: 70
R116-dzn-fzn
cavity_count: 32
R182-dzn-fzn
cavity_count: 70
A064-dzn-fzn
cavity_count: 162
R136-dzn-fzn
cavity_count: 16
R020-dzn-fzn
cavity_count: 68
A060-dzn-fzn
cavity_count: 100
R197-dzn-fzn
cavity_count: 49
A069-dzn-fzn
cavity_count: 186
A004-dzn-fzn
cavity_count: 4
R008-dzn-fzn
cavity_count: 18
R082-dzn-fzn
cavity_count: 12
R198-dzn-fzn
cavity_count: 50
R169-dzn-fzn
cavity_count: 67
R183-dzn-fzn
cavity_count: 70
R104-dzn-fzn
cavity_count: 16
R060-dzn-fzn
cavity_count: 54
A066-dzn-fzn
cavity_count: 170
R160-dzn-fzn
cavity_count: 80
A050-dzn-fzn
cavity_count: 70
R080-dzn-fzn
cavity_count: 28
A042-dzn-fzn
cavity_count: 76
R071-dzn-fzn
cavity_count: 6
R004-dzn-fzn
cavity_count: 36
R193-dzn-fzn
cavity_count: 104
A070-dzn-fzn
cavity_count: 190
A041-d

R083-dzn-fzn
cavity_count: 24
R065-dzn-fzn
cavity_count: 48
R203-dzn-fzn
cavity_count: 6
R079-dzn-fzn
cavity_count: 48
R108-dzn-fzn
cavity_count: 12
R187-dzn-fzn
cavity_count: 69


#### $Mz_2$

In [16]:
smt2_path = 'smt2-files/optimathsat/Mz2-noAbs'

for file in os.listdir(smt2_path):
    if file.endswith(".smt2"):
        adjust_smt2_file(smt2_path=smt2_path, file=file, write_path=f'''{smt2_path}/v4''')

R075-dzn-fzn
cavity_count: 10
R200-dzn-fzn
cavity_count: 48
R014-dzn-fzn
cavity_count: 20
A022-dzn-fzn
cavity_count: 80
A021-dzn-fzn
cavity_count: 80
R145-dzn-fzn
cavity_count: 24
R019-dzn-fzn
cavity_count: 64
R177-dzn-fzn
cavity_count: 70
R116-dzn-fzn
cavity_count: 32
R182-dzn-fzn
cavity_count: 70
A064-dzn-fzn
cavity_count: 162
R136-dzn-fzn
cavity_count: 16
R020-dzn-fzn
cavity_count: 68
A060-dzn-fzn
cavity_count: 100
R197-dzn-fzn
cavity_count: 49
A069-dzn-fzn
cavity_count: 186
A004-dzn-fzn
cavity_count: 4
R008-dzn-fzn
cavity_count: 18
R082-dzn-fzn
cavity_count: 12
R198-dzn-fzn
cavity_count: 50
R169-dzn-fzn
cavity_count: 67
R183-dzn-fzn
cavity_count: 70
R104-dzn-fzn
cavity_count: 16
R060-dzn-fzn
cavity_count: 54
A066-dzn-fzn
cavity_count: 170
R160-dzn-fzn
cavity_count: 80
A050-dzn-fzn
cavity_count: 70
R080-dzn-fzn
cavity_count: 28
A042-dzn-fzn
cavity_count: 76
R071-dzn-fzn
cavity_count: 6
R004-dzn-fzn
cavity_count: 36
R193-dzn-fzn
cavity_count: 104
A070-dzn-fzn
cavity_count: 190
A041-d

cavity_count: 36
R083-dzn-fzn
cavity_count: 24
R065-dzn-fzn
cavity_count: 48
R203-dzn-fzn
cavity_count: 6
R079-dzn-fzn
cavity_count: 48
R108-dzn-fzn
cavity_count: 12
R187-dzn-fzn
cavity_count: 69


## Test Generated `smt2` Files Using `OptiMathSAT`

This shoud generate the `smt2` file without any error. If this was the case then `OptiMathSAT` can be called on a file by running


```zsh
optimathsat output/A001-dzn-smt2-fzn.smt2 
```

yielding something similar to

```zsh
optimathsat output/A001-dzn-smt2-fzn.smt2                                                       
sat
(objectives
 (obj 41881)
)
(model 
  (define-fun X_INTRODUCED_981_ () Bool
    false)
  (define-fun X_INTRODUCED_348_ () Bool
    false)
   
   .....
```